In [ ]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/euler/substep_2_delta_1.0/')
gen=200_000
nb_genes_type = 20
gene_types = ['AB', 'A', 'B'] # Name of each gene type
gene_type_color = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
orient_name = ['leading', 'lagging'] # Name of each gene orientation
rel_orient = ['conv', 'div', 'upstr', 'downstr']

In [ ]:
def get_params(exp_path):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    with open(rep_dirs[0].joinpath('params.txt'), 'r') as params_file:
        param_lines = params_file.readlines()
        
    params = {}
    for line in param_lines:
        param_name = line.split(':')[0]
        if param_name == 'commit':
            param_val = line.split(':')[1].strip()
        else:
            param_val = float(line.split(':')[1])
        
        params[param_name] = param_val
        
    return params

In [ ]:
params = get_params(exp_path)

In [ ]:
params

In [ ]:
def get_best_indiv(rep_path, gen):
    
    with open(rep_path.joinpath(f'pop_gen_{gen:06}.evotsc'), 'rb') as save_file:
        pop_rep = pickle.load(save_file)
        
    pop_rep.evaluate()
    
    best_fit = 0
    best_indiv = pop_rep.individuals[0]
    
    try:
        for indiv in pop_rep.individuals:
            if indiv.fitness > best_fit:
                best_fit = indiv.fitness
                best_indiv = indiv
    except AttributeError: # In the neutral control, individuals are not evaluated so there is no fitness field
        pass
    
    return best_indiv

# Plot local supercoiling

In [ ]:
def get_sigma_tsc(indiv, sigma_env: float):
    temporal_expr = np.zeros((indiv.nb_genes, indiv.nb_eval_steps))

    # Initial values at t = 0
    temporal_expr[:, 0] = np.array([gene.basal_expression for gene in indiv.genes])
    
    sigma_tsc = np.zeros((indiv.nb_genes, indiv.nb_eval_steps))

    # Iterate the system
    for t in range(1, indiv.nb_eval_steps):
        sigma_local = indiv.inter_matrix @ temporal_expr[:, t-1]
        sigma_tsc[:, t] = sigma_local
        sigma_total = indiv.sigma_basal + sigma_local + sigma_env
        temporal_expr[:, t] = 1.0 / (1.0 + np.exp((sigma_total - indiv.sigma_opt)/indiv.epsilon))

    return sigma_tsc

In [ ]:
def plot_sigma_tsc(indiv, sigma_env, all_sigma=False, name=None):
    
    colors= ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
    
    sigma_tsc = get_sigma_tsc(indiv, sigma_env=sigma_env)
        
    plt.figure(figsize=(9, 6), dpi=300)
    for i_gene in range(indiv.nb_genes):
        tsc_val = sigma_tsc[i_gene]
        if all_sigma:
            tsc_val += indiv.sigma_basal + sigma_env
        plt.plot(tsc_val, color=colors[indiv.genes[i_gene].gene_type])
    
    
    # Add horizontal activation line
    x_min, x_max = plt.xlim()
    plt.hlines(params['sigma_opt'], x_min, x_max, linestyle='--', linewidth=1,
           color='tab:red', label='Half activation threshold')
    plt.xlim(x_min, x_max)

    plt.ylim(1, -1)
    
    plt.xlabel('Time')
    if all_sigma:
        plt.ylabel('Total supercoiling')
    else:
        plt.ylabel('TSC supercoiling')
    plt.title(f'Environment SC: {sigma_env:.2}')
    
    plt.grid(linestyle=':')
    
    if name:
        plt.savefig(name, dpi=300, bbox_inches='tight')
    
    plt.show()

In [ ]:
def plot_avg_sigma_tsc(indiv, sigma_env, all_sigma=False, title=None, name=None):
    
    colors = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
    
    sigma_tsc = get_sigma_tsc(indiv, sigma_env=sigma_env)
    
    avg_sigma_tsc = np.zeros((3, indiv.nb_eval_steps))
        
    fig, ax = plt.subplots(figsize=(9, 4), dpi=300)
    for i_gene in range(indiv.nb_genes):
        avg_sigma_tsc[indiv.genes[i_gene].gene_type] += sigma_tsc[i_gene]
    
    avg_sigma_tsc /= (indiv.nb_genes // 3)
    
    if all_sigma:
        avg_sigma_tsc += indiv.sigma_basal + sigma_env
        
    for i_gene_type, gene_type in enumerate(gene_types):
        ax.plot(avg_sigma_tsc[i_gene_type], color=colors[i_gene_type], label=f'{gene_type} genes')
    
    ax.set_xlabel('Time')
    
    # Add horizontal activation line
    x_min, x_max = ax.get_xlim()
    ax.hlines(params['sigma_opt'], x_min, x_max, linestyle='--', linewidth=1,
           color='tab:red', label='Half activation threshold')
    ax.set_xlim(x_min, x_max)

    
    if not all_sigma:
        ax.set_ylabel('TSC supercoiling')
    else:
        ax.set_ylabel('Total supercoiling')
        
    ax.set_ylim(0.25, -0.85)
    
    ax.grid(linestyle=':')
    ax.legend(loc='upper left')
    
    if title:
        ax.set_title(title)
    
    if name:
        plt.savefig(name, dpi=300, bbox_inches='tight')
    
    plt.show()

In [ ]:
def plot_med_sigma_tsc(indiv, sigma_env, all_sigma=False, title=None, name=None):
    
    colors = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
    
    sigma_tsc = get_sigma_tsc(indiv, sigma_env=sigma_env)
    
    sigma_tsc_type = {}
    
    for i_gene_type, gene_type in enumerate(gene_types):
        data = [sigma_tsc[i_gene, :] for i_gene in range(indiv.nb_genes)
                if indiv.genes[i_gene].gene_type == i_gene_type]
        sigma_tsc_type[gene_type] = pd.DataFrame(data)
        
        if all_sigma:
            sigma_tsc_type[gene_type] += indiv.sigma_basal + sigma_env
        

    fig, ax = plt.subplots(figsize=(9, 4), dpi=300)
    
    for i_gene_type, gene_type in enumerate(gene_types):
        med_sigma_tsc = sigma_tsc_type[gene_type].median()
        ax.plot(med_sigma_tsc, color=gene_type_color[i_gene_type], label=f'{gene_type} genes')
    
    ax.set_xlabel('Time')
    
    # Add horizontal activation line
    x_min, x_max = ax.get_xlim()
    ax.hlines(params['sigma_opt'], x_min, x_max, linestyle='--', linewidth=1,
           color='tab:red', label='Half activation threshold')
    ax.set_xlim(x_min, x_max)

    
    if not all_sigma:
        ax.set_ylabel('TSC supercoiling')
    else:
        ax.set_ylabel('Total supercoiling')
        
    ax.set_ylim(0.25, -0.85)
    
    ax.grid(linestyle=':')
    ax.legend(loc='upper left')
    
    if title:
        ax.set_title(title)
    
    if name:
        plt.savefig(name, dpi=300, bbox_inches='tight')
    
    plt.show()

In [ ]:
def plot_violin_sigma_tsc(indiv, sigma_env, wanted_type, all_sigma=False, title=None, name=None):
        
    sigma_tsc = get_sigma_tsc(indiv, sigma_env=sigma_env)
    
    sigma_tsc_type = {}
    
    for i_gene_type, gene_type in enumerate(gene_types):
        data = [sigma_tsc[i_gene, :] for i_gene in range(indiv.nb_genes)
                if indiv.genes[i_gene].gene_type == i_gene_type]
        sigma_tsc_type[gene_type] = pd.DataFrame(data)
        
        if all_sigma:
            sigma_tsc_type[gene_type] += indiv.sigma_basal + sigma_env

            
    fig, ax = plt.subplots(figsize=(9, 4), dpi=300)
    
    # On plotte que les gènes de type `wanted_type`
    for i_gene_type, gene_type in enumerate(gene_types):
        if gene_type != wanted_type:
            continue
    
        ax.plot(sigma_tsc_type[gene_type].median(), 
                #linestyle='',
                #marker='x',
                color=gene_type_color[i_gene_type], label=f'{gene_type} genes')
        
    #for i_gene_type, gene_type in enumerate(gene_types):
    ax.violinplot(sigma_tsc_type[wanted_type], positions=np.arange(indiv.nb_eval_steps),
                  showmedians=True) # A genes only for now
    
    ax.set_xlabel('Time')
    
    # Add horizontal activation line
    x_min, x_max = ax.get_xlim()
    ax.hlines(params['sigma_opt'], x_min, x_max, linestyle='--', linewidth=1,
           color='tab:red', label='Half activation threshold')
    ax.set_xlim(x_min, x_max)

    
    if not all_sigma:
        ax.set_ylabel('TSC supercoiling')
    else:
        ax.set_ylabel('Total supercoiling')
        
    ax.set_ylim(0.85, -0.85)
    
    if title:
        ax.set_title(title)
    ax.grid(linestyle=':')
    ax.legend(loc='upper left')
    
    if name:
        plt.savefig(name, dpi=300, bbox_inches='tight')
    
    plt.show()

In [ ]:
def plot_best_sigma_tsc(exp_path, gen):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    for rep, rep_dir in enumerate(rep_dirs):

        best_indiv = get_best_indiv(rep_dir, gen)
                
        plot_violin_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], wanted_type='A', all_sigma=True,
                              title='Environment A: A genes',
                              name=exp_path.joinpath(f'violin_sc_rep{rep:02}_env_A_genes_A.pdf'))
        plot_violin_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], wanted_type='B', all_sigma=True,
                              title='Environment A: B genes',
                              name=exp_path.joinpath(f'violin_sc_rep{rep:02}_env_A_genes_B.pdf'))
        plot_violin_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], wanted_type='AB', all_sigma=True,
                              title='Environment A: AB genes',
                              name=exp_path.joinpath(f'violin_sc_rep{rep:02}_env_A_genes_AB.pdf'))

        
        plot_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], all_sigma=True,
                       name=exp_path.joinpath(f'sigma_tsc_best_rep{rep:02}_env_A.pdf'))
    
        plot_sigma_tsc(best_indiv, sigma_env=params['sigma_B'], all_sigma=True,
                       name=exp_path.joinpath(f'sigma_tsc_best_rep{rep:02}_env_B.pdf'))

        plot_avg_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], title='Environment A: mean SC',
                           name=exp_path.joinpath(f'sigma_tsc_avg_best_rep{rep:02}_env_A_mean.pdf'))
        plot_med_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], title='Environment A: median SC',
                           name=exp_path.joinpath(f'sigma_tsc_avg_best_rep{rep:02}_env_A_med.pdf'))

        plot_avg_sigma_tsc(best_indiv, sigma_env=params['sigma_B'],
                           name=exp_path.joinpath(f'sigma_tsc_avg_best_rep{rep:02}_env_B.pdf'))
        
        plot_avg_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], all_sigma=True,
                           title='Environment A: mean SC',
                           name=exp_path.joinpath(f'sigma_tsc_all_best_rep{rep:02}_env_A_mean.pdf'))
        plot_med_sigma_tsc(best_indiv, sigma_env=params['sigma_A'], all_sigma=True,
                           title='Environment A: median SC',
                           name=exp_path.joinpath(f'sigma_tsc_all_best_rep{rep:02}_env_A_med.pdf'))

        plot_avg_sigma_tsc(best_indiv, sigma_env=params['sigma_B'], all_sigma=True, title='Environment B',
                           name=exp_path.joinpath(f'sigma_tsc_all_best_rep{rep:02}_env_B.pdf'))


In [ ]:
plot_best_sigma_tsc(exp_path, gen)